# NYC taxi visualization

In this tutorial, we were provided with a dataset which contains information about the number of taxi ride drop-offs in New York City in the year 2015.
The raw data (obtained from the [Vaex documentation](https://vaex.readthedocs.io/en/latest/datasets.html))
has been histogrammed into three dimensions: latitude, longitude and hour-of-the-day.

We will first load the data and inspect its contents:

In [ ]:
%matplotlib widget
import scipp as sc
import plopp as pp

data = sc.io.open_hdf5('nyc_taxi_data.h5')
data

The goal of this tutorial is to create an interface with the following:

- a central 2D image that shows a longitude-latitude map of the drop-offs, for a single hour-of-the-day slice.
- a slider below the main panel which allows to navigate the hour-of-the-day dimension, updating the 2D map.
- a 1D figures above below the 2D map, displaying the same data but summed along the latitude dimension.
- in the 1D panel, we also want to add a gaussian smoothed version of the 1D curve.

The first step is to create an `input_node` for the original data,
a `widget_node` that will slice the `hour` dimension,
and a `sliced` node that slices the original data according to the index of the slider.

In [ ]:
# Input data node
a = pp.input_node(data)

# Slider node that provides index to slice
sl = pp.widgets.SliceWidget(data, ['hour'])
w = pp.widget_node(sl)

# Node that actually does the slicing
sliced = pp.widgets.slice_dims(a, w)

pp.show_graph(a)

Next, we add a two-dimensional figure to the bottom node that will display the latitude/longitude map:

In [ ]:
# Add figure to the bottom node
fig = pp.figure2d(sliced, norm='log')
pp.show_graph(a)

We can now add a node that performs sum along the latitude dimension

In [ ]:
# Add a node that performs sum along the latitude dimension
sumx = node(sc.sum, dim='latitude')(sliced)
show_graph(a)

We next add a smoothing node that will smooth the 1d data below the `sliced` node

In [ ]:
# Add a smoothing node that will smooth the 1d data
smooth = node(gaussian_filter)(sumx, sigma=input_node(5))
show_graph(a)

Finally, we add the bottom panel figure, giving it both the `sumx` and `smooth` nodes as inputs,
to display both curves on the same axes:

In [ ]:
# Add figure that plots both smoothed and unsmoothed data
fx = figure(sumx, smooth, norm='log')
show_graph(a)

### Putting it all together

In [ ]:
a = pp.input_node(data)  # Input data node

# Slider node that provides index to slice
sl = pp.widgets.SliceWidget(data, ['hour'])
w = pp.widget_node(sl)

sliced = pp.node(lambda da, ind: da['hour', ind])(da=a, ind=w)  # Node that slices the data

fig2d = pp.figure2d(sliced, norm='log')  # Figure showing 2d image

sumx = pp.node(sc.sum, dim='latitude')(sliced)  # Sum along the latitude dimension

smooth = pp.node(gaussian_filter)(sumx, sigma=pp.input_node(5))  # Gaussian smoothing

fig1d = pp.figure1d(sumx, smooth, norm='log')  # Two lines on the same figure

pp.widgets.Box([sl, fig2d, fig1d])  # Container box

Now when we interact with the slider, the 2D map, as well as both curves on the 1D profile will update.